This script can be used to generate the following events given some KPI constarints for one palnned date for one particular line:

*   FTT
*   Defective
*   Defects
*   Reject
*   Rectified
*   Manpower
*   Reject reasons

The inputs we need are:

1. A plan for one day/ multipe days  for the line of interest
2. An hourly table of KPIs for each day (date column and hour column both)
3. General schedule (Assuming we reuse the same schedule for all days)
4. An example etmplate for each event type (just to order the columns)

We have all 4 sheets in one file  above in the link here which we will upload to **content/sample_data**
[link text](https://docs.google.com/spreadsheets/d/1sX9sf6zxVHKcaqhAdU-L36pvrUDtaOqK/edit?usp=drive_link)

 5. Synthesizers : Dump of all the pretrained synthesizers needed to generate  
    the constrained synthetic data.
    **(Upload to content/sample_data when running the script)**
[link text](https://drive.google.com/drive/folders/156GDl2-paytduo3noFF55BPhQzvFjBD_?usp=drive_link)





The output files per attribute will be dumped under **content/** using the attribute name in .csv format.




   









In [ ]:
# All imports
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import string
import uuid

In [ ]:
# Read constraints sheet
xls = pd.read_excel("/content/sample_data/constraints_mul_days.xlsx",
                    sheet_name=["plan","schedule", "target_kpis",
                                "ftt", "defective", "reject", "defects",
                                "rectified", "manpower", "rr"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]
target_kpis = xls["target_kpis"]
ftt = xls["ftt"]
defective = xls["defective"]
reject = xls["reject"]
defects = xls["defects"]
rectified = xls["rectified"]
manpower = xls["manpower"]
reject_reason = xls["rr"]

In [ ]:
# Timezone of the factory
hour_offset = 5.5

I assume that the plan is uploaded per-day (can have multiple plans though)

In [ ]:
# Function to generate random timestamps per-day that lies between a given start and end times

import random
import pandas as pd
from datetime import datetime, timedelta

def generate_random_timestamps(start_time, end_time, min_increment, max_increment, num_increments):
    """
    Generate a sorted list of timestamps with random increments for a specified date and time range.

    :param start_time: The start datetime.
    :param end_time: The end datetime.
    :param min_increment: Minimum increment in minutes.
    :param max_increment: Maximum increment in minutes.
    :param num_increments: Number of random increments to generate.
    :return: A DataFrame with the sorted timestamps.
    """
    # Generate random increments
    increments = [random.randint(min_increment, max_increment) for _ in range(num_increments)]

    # Generate timestamps
    timestamps = []
    current_time = start_time

    while current_time < end_time:
        timestamps.append(current_time)
        increment = timedelta(seconds=random.choice(increments))
        current_time += increment
        if current_time > end_time:
            break

    # Ensure timestamps are sorted
    timestamps.sort()

    # Create a DataFrame
    df = pd.DataFrame({'ts': timestamps})

    return df

In [ ]:
all_dates = plan["date"].to_list()

In [ ]:
# Logic to determine function parameters(with an offset for factory loacl time)

date_list =[]
start_time_list = []
end_time_list = []
num_increments_list = []

for index in range(len(all_dates)):
  date = all_dates[index].to_pydatetime().date()
  start_time_str = pd.to_timedelta(schedule["start_time"].iloc[0].strftime('%H:%M:%S'))- pd.Timedelta(hours=hour_offset)
  end_time_str = pd.to_timedelta(schedule["end_time"].iloc[-1].strftime('%H:%M:%S'))- pd.Timedelta(hours=hour_offset)

  start_time = datetime.combine(date, (datetime.min + start_time_str).time())
  end_time = datetime.combine(date, (datetime.min+ end_time_str).time())

  num_increments = plan["qty"].iloc[index] + random.randint(-1, 1)
  print(date)
  print(start_time)
  print(end_time)
  print(num_increments)

  date_list.append(date)
  start_time_list.append(start_time)
  end_time_list.append(end_time)
  num_increments_list.append(num_increments)



  #num_increments = 1000

min_increment = 2
max_increment = 6

2024-07-26
2024-07-26 02:00:00
2024-07-26 13:30:00
1101
2024-07-27
2024-07-27 02:00:00
2024-07-27 13:30:00
999
2024-07-28
2024-07-28 02:00:00
2024-07-28 13:30:00
799
2024-07-29
2024-07-29 02:00:00
2024-07-29 13:30:00
1001
2024-07-30
2024-07-30 02:00:00
2024-07-30 13:30:00
901
2024-07-31
2024-07-31 02:00:00
2024-07-31 13:30:00
1000
2024-08-01
2024-08-01 02:00:00
2024-08-01 13:30:00
1199


In [ ]:
# Verification
print(start_time_list)


[datetime.datetime(2024, 7, 26, 2, 0), datetime.datetime(2024, 7, 27, 2, 0), datetime.datetime(2024, 7, 28, 2, 0), datetime.datetime(2024, 7, 29, 2, 0), datetime.datetime(2024, 7, 30, 2, 0), datetime.datetime(2024, 7, 31, 2, 0), datetime.datetime(2024, 8, 1, 2, 0)]


**FTT**

In [ ]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)
ts_df = pd.DataFrame()

for i in range(len(date_list)):
  ts_df= pd.concat([ts_df, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_df.reset_index(drop=True, inplace=True)
ts_df

,ts
0,2024-07-26 02:00:00
1,2024-07-26 02:00:06
2,2024-07-26 02:00:11
3,2024-07-26 02:00:13
4,2024-07-26 02:00:15
...,...
72565,2024-08-01 13:29:43
72566,2024-08-01 13:29:46
72567,2024-08-01 13:29:49
72568,2024-08-01 13:29:53


Now, I assume we have the trained synthesizers for each event type given below which we can load and use to generate more samples. But we may have to alter categorical data column values according to the plan's bctx and org. (So just learned neumerical distributions are learned.)

In [ ]:
!pip install sdv

In [ ]:
# To import a synthesizer
import sdv
from sdv.single_table import CTGANSynthesizer

In [ ]:
# Import the pretrained synthesizer for FTT
ftt_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/ftt_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# Generate some instances (to teh same length as timestamps)
synthetic_ftt = ftt_synthesizer.sample(num_rows=len(ts_df))

Sampling rows: 100%|██████████| 72570/72570 [00:07<00:00, 10194.57it/s]


In [ ]:
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 12:33:28,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-06-01 12:57:38,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-06-01 06:52:09,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-06-01 06:28:48,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-06-01 08:23:40,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0


In [ ]:
# Function to determine the date based on a time threshold
def get_factory_date(dt):
    dt = dt + timedelta(hours=hour_offset)
    if dt.time() < pd.to_datetime('05:00:00').time():
        return (dt - timedelta(days=1)).date()
    else:
        return dt.date()

In [ ]:
# Apply factory_date logic to evt_gmt
synthetic_ftt["evt_gmt"]= ts_df["ts"]

synthetic_ftt['evt_gmt'] = pd.to_datetime(synthetic_ftt['evt_gmt'])
synthetic_ftt['factory_date'] = synthetic_ftt['evt_gmt'].apply(get_factory_date)

In [ ]:
# Verification
synthetic_ftt

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-07-26 02:00:06,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-07-26 02:00:11,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-07-26 02:00:15,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72565,2024-08-01 13:29:43,2024-08-01,sdv-id-YdnzYs,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,6,0,0,0,0,0,0
72566,2024-08-01 13:29:46,2024-08-01,sdv-id-lddpEY,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,6,0,0,0,0,0,0
72567,2024-08-01 13:29:49,2024-08-01,sdv-id-KiUylR,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,8,0,0,0,0,0,0
72568,2024-08-01 13:29:53,2024-08-01,sdv-id-uAccxw,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,9,0,0,0,0,0,0


In [ ]:
def edit_df_bctx_org(df,columns):
    # Ensure date_column is in datetime format
    df["factory_date"] = pd.to_datetime(df["factory_date"])
    plan["date"] = pd.to_datetime(plan["date"])

    # Iterate through each row in the df
    for i in range(len(df)):
        # Get the date for the current row in df
        current_date = df.loc[i, "factory_date"]

        # Find the corresponding row in the plan DataFrame
        plan_row = plan[plan["date"] == current_date]

        if not plan_row.empty:
            for column in columns:
                df.at[i, column] = plan_row.iloc[0][column]
        else:
            print(f"Date {current_date} not found in plan DataFrame")

    return df

In [ ]:
# For the ease of manipulation, let us assume that we add piece by piece in
# all scenarios
# This functions sets a value column to a given value

def edit_value(df, column, value):
  df[column] = value

In [ ]:
edit_value(synthetic_ftt, "ftt_count", 1)

In [ ]:
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
1,2024-07-26 02:00:06,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
2,2024-07-26 02:00:11,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
4,2024-07-26 02:00:15,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0


In [ ]:
def find_working_hour(column, hours):
  local_time = column + timedelta(hours=hours)
  return local_time.hour * 3600 + local_time.minute * 60 + local_time.second


In [ ]:
synthetic_ftt['factory_time'] = synthetic_ftt['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27000
1,2024-07-26 02:00:06,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,27006
2,2024-07-26 02:00:11,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27011
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,27013
4,2024-07-26 02:00:15,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27015


In [ ]:
# Add the seconds of the day to the schedule df
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:00').dt.time
schedule['seconds_of_day'] = schedule['start_time'].apply(lambda x: x.hour * 3600 + x.minute * 60)

In [ ]:
# Verification
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [ ]:
# Perform a forward-looking merge
synthetic_ftt = synthetic_ftt.sort_values('factory_time')
synthetic_ftt = pd.merge_asof(synthetic_ftt, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_ftt.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)



In [ ]:
# Verification
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,hour
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,1
1,2024-07-29 02:00:00,2024-07-29,sdv-id-mKMTrh,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
2,2024-07-31 02:00:00,2024-07-31,sdv-id-Jwohhp,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
3,2024-07-28 02:00:00,2024-07-28,sdv-id-OetnIQ,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
4,2024-08-01 02:00:00,2024-08-01,sdv-id-GPkJfj,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1


In [ ]:
synthetic_ftt = synthetic_ftt.rename(columns={'hour': 'Hour', 'factory_date': 'Date'})


In [ ]:
target_kpis

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-07-26,MLF1001,1,91,8.28,1.666667,9,13,3,0,23,5,28
1,2024-07-26,MLF1001,2,95,8.28,1.666667,5,7,4,2,23,5,28
2,2024-07-26,MLF1001,3,95,8.28,1.666667,6,12,2,2,23,5,28
3,2024-07-26,MLF1001,4,92,8.28,1.666667,1,1,1,1,23,5,28
4,2024-07-26,MLF1001,5,93,8.28,1.666667,5,9,3,2,23,5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01,MLF1001,7,110,8.28,1.820000,2,3,2,0,26,8,34
73,2024-08-01,MLF1001,8,100,8.28,1.820000,2,3,1,0,26,8,34
74,2024-08-01,MLF1001,9,106,8.28,1.820000,1,2,1,1,26,8,34
75,2024-08-01,MLF1001,10,104,8.28,1.820000,1,2,1,1,26,8,34


In [ ]:
# Function to extract only the necessary attribute targets
def get_relevant_target(columns):
  targets = target_kpis[columns]
  return targets

In [ ]:
# Traget for FTTs
targets_ftt = get_relevant_target(["Date","Hour", "FTT"])

In [ ]:
def adjust_hourly_counts(df, target_counts, count_column, hour_column='Hour', date_column='Date'):
    """
    Adjust the counts in each hour and date group of a DataFrame to match the target counts.

    :param df: DataFrame containing hour, date, and count columns.
    :param target_counts: DataFrame with hour, date, and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with adjusted counts.
    """

    # Ensure date and hour columns are of the same type (datetime64[ns])
    df[date_column] = pd.to_datetime(df[date_column])  # Convert
    df[hour_column] = pd.to_datetime(df[hour_column], format='%H').dt.hour

    # Merge the dataframes on the 'hour' and 'date' columns
    merged_df = df.merge(target_counts, on=[hour_column, date_column], how='left')

    # Create a list to store the adjusted groups
    result_groups = []

    # Iterate over each hour and date group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Calculate the difference between the current count and the target count
        current_count = len(group)
        target_count = group[count_column].iloc[0]
        difference = current_count - target_count

        # Drop rows randomly to meet the target count
        if difference > 0:
            drop_indices = group.sample(n=difference).index
            group = group.drop(drop_indices)

        # Append the group to the result list
        result_groups.append(group)

    # Concatenate the adjusted groups into a single DataFrame
    result_df = pd.concat(result_groups, ignore_index=True)

    # Drop the 'target_count' column as it's no longer needed
    #result_df = result_df.drop(columns=[count_column])

    return result_df




In [ ]:
synthetic_ftt_regulated = adjust_hourly_counts(synthetic_ftt, targets_ftt,"FTT", hour_column='Hour')
synthetic_ftt_regulated = synthetic_ftt_regulated.reindex(columns=ftt.columns)
synthetic_ftt_regulated['factory_date'] = synthetic_ftt_regulated['evt_gmt'].apply(get_factory_date)


In [ ]:
synthetic_ftt_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'ftt_count', 'defective_count',
       'defect_count', 'reject_count', 'rectified_count', 'input_count',
       'rw_defective_count'],
      dtype='object')

In [ ]:
# # Function to add unit_id, unit columns

# def add_unit_columns(df):
#     # Get the index of the 'department_id' column
#     department_index = df.columns.get_loc('department_id')

#     # Insert 'unit_id' column with null values before 'department_id'
#     df.insert(department_index, 'unit_id', [None] * len(df))

#     # Insert 'unit' column with null values before 'department_id' (again since 'department_id' index has shifted)
#     df.insert(department_index, 'unit', [None] * len(df))

#     return df

In [ ]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory","unit_id", "unit", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "department_id","department", "module_id",
                        "module", "stp", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

edit_df_bctx_org(synthetic_ftt_regulated, editable_columns)

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:06,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
1,2024-07-26 02:01:07,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
2,2024-07-26 02:01:23,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
3,2024-07-26 02:01:50,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
4,2024-07-26 02:02:23,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,2024-08-01 13:26:37,2024-08-01,12f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
6961,2024-08-01 13:27:10,2024-08-01,12f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,1,0,0,0,0,0,0
6962,2024-08-01 13:27:23,2024-08-01,12f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,1,0,0,0,0,0,0
6963,2024-08-01 13:29:13,2024-08-01,12f0ffe7-c509-49b5-b203-83be908b2887,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,1,0,0,0,0,0,0


In [ ]:
# Change the line_id manually

synthetic_ftt_regulated['stp'] = synthetic_ftt_regulated['stp'] +  '-station2'
#synthetic_ftt_regulated['stp']


In [ ]:
#synthetic_ftt_regulated.to_csv('synthetic_ftt_regulated.csv', index=False)

**DEFECTIVES**

In [ ]:
# Generate a large number of timestamps
ts_defective = pd.DataFrame()

for i in range(len(date_list)):
  ts_defective= pd.concat([ts_defective, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_defective.reset_index(drop=True, inplace=True)
ts_defective.head()

,ts
0,2024-07-26 02:00:00
1,2024-07-26 02:00:04
2,2024-07-26 02:00:08
3,2024-07-26 02:00:13
4,2024-07-26 02:00:17


In [ ]:
# Import the pretrained synthesizer for defective
defective_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/defective_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# Generate some instances (to the same length as timestamps)
synthetic_defective = defective_synthesizer.sample(num_rows=len(ts_defective))

Sampling rows: 100%|██████████| 72477/72477 [00:10<00:00, 6896.01it/s]


In [ ]:
# Apply factory_date logic to evt_gmt
synthetic_defective["evt_gmt"]= ts_defective["ts"]

synthetic_defective['evt_gmt'] = pd.to_datetime(synthetic_defective['evt_gmt'])
synthetic_defective['factory_date'] = synthetic_defective['evt_gmt'].apply(get_factory_date)

In [ ]:
# Edit defective value

edit_value(synthetic_defective, "defective_count", 1)

In [ ]:
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-wnVsWx,0,1,1,0,0,0,0
1,2024-07-26 02:00:04,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-GrLoVq,0,1,1,0,0,0,0
2,2024-07-26 02:00:08,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-HRikVU,0,1,1,0,0,0,0
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-DgKEuo,0,1,3,0,0,0,0
4,2024-07-26 02:00:17,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,sdv-id-AQYBhy,0,1,1,0,0,0,0


In [ ]:
# Function to generate gid
def generate_random_string(length=8):
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

In [ ]:
# Generate gid
synthetic_defective["gid"]= [generate_random_string() for _ in range(len(synthetic_defective))]

In [ ]:
# Drop rows that may have generated the same random gid (rare but a precaution)
synthetic_defective= synthetic_defective.drop_duplicates(subset=['gid'])

In [ ]:
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,ILLCTIXI,0,1,1,0,0,0,0
1,2024-07-26 02:00:04,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,J3OVC59W,0,1,1,0,0,0,0
2,2024-07-26 02:00:08,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,5O1BN7RP,0,1,1,0,0,0,0
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,7Y39OU46,0,1,3,0,0,0,0
4,2024-07-26 02:00:17,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,LE9PX2CD,0,1,1,0,0,0,0


In [ ]:
# Convert timestamp to seconds of day
synthetic_defective['factory_time'] = synthetic_defective['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,defective,ILLCTIXI,0,1,1,0,0,0,0,27000
1,2024-07-26 02:00:04,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,defective,J3OVC59W,0,1,1,0,0,0,0,27004
2,2024-07-26 02:00:08,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,defective,5O1BN7RP,0,1,1,0,0,0,0,27008
3,2024-07-26 02:00:13,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,defective,7Y39OU46,0,1,3,0,0,0,0,27013
4,2024-07-26 02:00:17,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,defective,LE9PX2CD,0,1,1,0,0,0,0,27017


In [ ]:
# Perform a forward-looking merge
synthetic_defective = synthetic_defective.sort_values('factory_time')
synthetic_defective = pd.merge_asof(synthetic_defective, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_defective.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
synthetic_defective = synthetic_defective.rename(columns={'hour': 'Hour', 'factory_date': 'Date' })
synthetic_defective.head()



,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,defective,ILLCTIXI,0,1,1,0,0,0,0,1
1,2024-07-29 02:00:00,2024-07-29,sdv-id-rfoDEe,1,Morning,1,mas,1,kreeda,ABC001,...,defective,YJAPFBD0,0,1,1,0,0,0,0,1
2,2024-07-31 02:00:00,2024-07-31,sdv-id-tkSqyl,1,Morning,1,mas,1,kreeda,ABC001,...,defective,3UL9FL23,0,1,2,0,0,0,0,1
3,2024-07-28 02:00:00,2024-07-28,sdv-id-PxldTc,1,Morning,1,mas,1,kreeda,ABC001,...,defective,HMAAEILB,0,1,2,0,0,0,0,1
4,2024-08-01 02:00:00,2024-08-01,sdv-id-RluMlR,1,Morning,1,mas,1,kreeda,ABC001,...,defective,UE10NMWD,0,1,1,0,0,0,0,1


In [ ]:
# Target for defectives
targets_defective = get_relevant_target(["Date","Hour", "Defective", "Defect"])
targets_defective

,Date,Hour,Defective,Defect
0,2024-07-26,1,9,13
1,2024-07-26,2,5,7
2,2024-07-26,3,6,12
3,2024-07-26,4,1,1
4,2024-07-26,5,5,9
...,...,...,...,...
72,2024-08-01,7,2,3
73,2024-08-01,8,2,3
74,2024-08-01,9,1,2
75,2024-08-01,10,1,2


In [ ]:
#pd.set_option('display.max_rows', None)

In [ ]:
# Adjust the defective count to tally with the given target
defective_adjusted = adjust_hourly_counts(synthetic_defective, targets_defective,
                                          "Defective", hour_column="Hour")
defective_adjusted.head(20)
#defective_adjusted_final = defective_adjusted.drop(columns=['Hour', 'Defective',
                                                            #'Defect'])

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour,Defective,Defect
0,2024-07-26 02:05:34,2024-07-26,sdv-id-oKBdBP,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
1,2024-07-26 02:09:28,2024-07-26,sdv-id-ZdVGji,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
2,2024-07-26 02:14:40,2024-07-26,sdv-id-NCVvba,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
3,2024-07-26 02:18:00,2024-07-26,sdv-id-KcfnFF,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
4,2024-07-26 02:21:37,2024-07-26,sdv-id-PhTqRM,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
5,2024-07-26 02:34:48,2024-07-26,sdv-id-pDJYOy,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
6,2024-07-26 02:45:31,2024-07-26,sdv-id-CmcBnM,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
7,2024-07-26 02:46:58,2024-07-26,sdv-id-IHucmu,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
8,2024-07-26 02:51:31,2024-07-26,sdv-id-slBuqK,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
9,2024-07-26 03:00:12,2024-07-26,sdv-id-cWriKQ,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,2,5,7


In [ ]:
# Dataframe we can use to regulate defect counts
defective_adjusted.drop(columns=['Defective','Defect'], inplace=True)

In [ ]:
def distribute_defects(total_defects, num_defectives):
  """
    Distribute a given number of total defects between a specified number
    of defective events.

    :param total_sum: The total defects to be distributed.
    :param num_events: The number of defective events.
    :return: A list with the distributed values.
  """
  # Initial distribution: assign each event a base value

  base_defects = total_defects // num_defectives
  remaining = total_defects % num_defectives

  distribution = [base_defects] * num_defectives

  # Randomly distribute the remaining value
  for _ in range(remaining):
      distribution[np.random.randint(0, num_defectives)] += 1

  return distribution



In [ ]:
def generate_hourly_counts(defectives, targets, hour_column='Hour',date_column= 'Date', count_column='defect_count'):
    """
    Generate counts for each hour of each day group of a DataFrame to match the target counts by
    distributing the target sum among all rows in the hour + date group.

    :param defectives: DataFrame containing hour and count columns.
    :param targets: DataFrame with hour and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with generated counts.
    """
    # Merge the dataframes on the 'hour and date' column
    merged_df = defectives.merge(targets, on=[date_column, hour_column], how='left')

    # Create a new column for generated counts
    merged_df['generated_count'] = 0

    # Iterate over each hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target sum for this date + hour
        target_sum = group['Defect'].iloc[0]

        # Generate the new counts by distributing the target sum
        generated_counts = distribute_defects(target_sum, len(group))
        merged_df.loc[group.index, 'generated_count'] = generated_counts

    # # Drop the 'target_count' column as it's no longer needed
    # result_df = merged_df.drop(columns=['target_count'])

    return merged_df


In [ ]:
merged_df = generate_hourly_counts(defective_adjusted, targets_defective)
merged_df['defect_count']= merged_df['generated_count']



In [ ]:
merged_df= merged_df.rename(columns={'Date': 'factory_date'})

In [ ]:
# Function to add unit_id, unit columns

def add_unit_columns(df):
    # Get the index of the 'department_id' column
    department_index = df.columns.get_loc('department_id')

    # Insert 'unit_id' column with null values before 'department_id'
    df.insert(department_index, 'unit', [None] * len(df))

    # Insert 'unit' column with null values before 'department_id' (again since 'department_id' index has shifted)
    df.insert(department_index, 'unit_id', [None] * len(df))

    return df

In [ ]:
merged_df = add_unit_columns(merged_df)

In [ ]:
merged_df.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'gid', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count', 'Hour', 'Defective', 'Defect',
       'generated_count'],
      dtype='object')

In [ ]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory","unit_id", "unit", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone","department_id","department", "section_id",
                        "section", "module_id","module", "stp",  "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

synthetic_defective_regulated = edit_df_bctx_org(merged_df, editable_columns)

defects_for_defect = synthetic_defective_regulated.drop(columns=['generated_count','Defective', 'Defect'])


merged_df = synthetic_defective_regulated.drop(columns=['generated_count','Hour', 'Defective', 'Defect'])

#merged_df.to_csv('synthetic_defective_regulated.csv', index=False)

In [ ]:
# Change the line_id manually
merged_df['stp'] = merged_df['stp'] +  '-station2'

In [ ]:
synthetic_defective_regulated = merged_df.copy()


In [ ]:
synthetic_defective_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'gid', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count'],
      dtype='object')

**DEFECTS**

In [ ]:
# To create the defects dataframe, explode each of the  events in defective data,
# defect_count times
def generate_defects_base(columns):
  df_repeated = pd.DataFrame()
  for col in columns:
    repeated_df = []
    for index, row in merged_df.iterrows():
        repeated_df.extend([row[col]] * row['defect_count'])
    df_repeated[col] = repeated_df
  return df_repeated

In [ ]:
columns_for_defects = ["evt_gmt",	"factory_date",	"shift_id",	"shift",
                        "tenant_id",	"tenant",	"cluster_id",	"cluster",
                        "factory_id",	"factory",	"building_id",	"building",
                        "unit_id", "unit","department_id",	"department",
                        "floor_id",	"floor","zone_id",	"zone",	"section_id",
                         "section",	"module_id","module","stp","size",	"hsk",
                        "buf","apt","stpv","buyer_id","buyer","vendor_id",
                        "vendor","po_id","po","so_id","so","li_id","li",
                        "fabric_type_id",	"fabric_type",	"product_category_id",
                        "product_category",	"product_type_id",	"product_type",
                        "style_id",	"style","color_id",	"color","size_id","gid"]
defects_base = generate_defects_base(columns_for_defects)

In [ ]:
# Findout already processed columns
added_columns = defects_base.columns

In [ ]:
# Findout what other columns should be in defects data

for column_name in added_columns:
    defects_temp = defects.drop(column_name, axis=1)
defects_temp.head(200)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,issue_color_code,issue,operation_id,operation_code,operation,image,silhouette_id,silhouette,x,y
0,2024-06-01 02:15:00,2024-06-01 08:12:15,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,0.76,0.49
1,2024-06-02 02:16:00,2024-06-01 08:33:59,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#808080,Bobbling,18,117,Att Match Bow at Waist,front,1,shirt_style1,0.73,0.59
2,2024-06-02 02:16:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#FFFFCC,Hi Low,11,110,Att Elastic at Both Legs,back,1,shirt_style1,1.00,0.07
3,2024-06-03 02:34:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,7,106,BT at Waist W /Trim * 1,front,1,shirt_style1,1.00,0.75
4,2024-06-04 02:35:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,1.00,0.00
5,2024-06-05 02:41:00,2024-06-01 08:32:28,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800080,Wavy,11,110,Att Elastic at Both Legs,front,1,shirt_style1,0.84,0.45
6,2024-06-06 02:43:00,2024-06-01 08:32:28,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#000080,Pleat,17,116,Att Bow at CF Waist W / mark,back,1,shirt_style1,0.74,0.00
7,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#660066,Down Stc,11,110,Att Elastic at Both Legs,back,1,shirt_style1,1.00,0.64
8,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#993366,Oil Spot,14,113,Att FO Elastic Around Waist,front,1,shirt_style1,0.87,0.45
9,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,7,106,BT at Waist W /Trim * 1,front,1,shirt_style1,0.77,0.40


In [ ]:
# Import the pretrained synthesizer for defective
defects_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/defects_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# Generate synthetic dataset that is the same length as aggregated defective data
synthetic_defects = defects_synthesizer.sample(num_rows=len(defects_base))

Sampling rows: 100%|██████████| 343/343 [00:00<00:00, 3452.67it/s]


In [ ]:
#Concatenate the aggregated defective data with generated defect-specific data
synthetic_defects = pd.concat([defects_base, synthetic_defects], axis=1)

In [ ]:
# Manually change event type
synthetic_defects["evn"]= "defect"

In [ ]:
# Reorder the data columns
synthetic_defects = synthetic_defects.reindex(columns=defects.columns)
synthetic_defects.head(20)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,issue_color_code,issue,operation_id,operation_code,operation,image,silhouette_id,silhouette,x,y
0,2024-07-26 02:05:34,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#CCCCFF,Join Stc,11,110,Att Elastic at Both Legs,front,1,shirt_style1,0.98,0.35
1,2024-07-26 02:09:28,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#808000,Steps,11,110,Att Elastic at Both Legs,back,1,shirt_style1,0.52,1.00
2,2024-07-26 02:09:28,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#993366,Oil Spot,10,109,Join Bk Gusset Seam,back,1,shirt_style1,0.77,0.59
3,2024-07-26 02:14:40,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#FF00FF,Reverse,12,111,Join 1st Side Seam,front,1,shirt_style1,0.75,0.63
4,2024-07-26 02:18:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,0.92,0.75
5,2024-07-26 02:18:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#0066CC,Bow Defect,12,111,Join 1st Side Seam,back,1,shirt_style1,0.86,0.40
6,2024-07-26 02:21:37,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#993366,Oil Spot,14,113,Att FO Elastic Around Waist,back,1,shirt_style1,0.82,0.24
7,2024-07-26 02:21:37,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800080,Wavy,10,109,Join Bk Gusset Seam,back,1,shirt_style1,0.45,0.20
8,2024-07-26 02:34:48,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,1.00,0.73
9,2024-07-26 02:45:31,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#008080,Bartack,1,100,Join Fnt Gusset Seam,front,1,shirt_style1,0.69,0.13


In [ ]:
synthetic_defects.columns

Index(['evt_gmt', 'factory_date', 'shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department',
       'floor_id', 'floor', 'zone_id', 'zone', 'section_id', 'section',
       'module_id', 'module', 'station_id', 'station', 'stp', 'size', 'hsk',
       'buf', 'apt', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'sign', 'gid', 'issue_id', 'issue_code',
       'issue_color_code', 'issue', 'operation_id', 'operation_code',
       'operation', 'image', 'silhouette_id', 'silhouette', 'x', 'y'],
      dtype='object')

In [ ]:
#synthetic_defects.to_csv('sythetic_defects_regulated.csv', index=False)

**RECTIFIED**

In [ ]:
# We are going to derive rectified as portion of events from the defectives
# with the working hour attached.
defectives = defects_for_defect.copy()
defectives.head()
defectives.rename(columns={'factory_date': 'Date'}, inplace=True)

In [ ]:
# Target for rectified
targets_rectified = get_relevant_target(["Date", "Hour", "Rectified"])
targets_rectified

,Date,Hour,Rectified
0,2024-07-26,1,3
1,2024-07-26,2,4
2,2024-07-26,3,2
3,2024-07-26,4,1
4,2024-07-26,5,3
...,...,...,...
72,2024-08-01,7,2
73,2024-08-01,8,1
74,2024-08-01,9,1
75,2024-08-01,10,1


In [ ]:
def pick_rectified_events(all_defective, targets, hour_column='Hour',date_column = 'Date', datetime_column='evt_gmt', count_column='Rectified'):
    """
    Pick the first n events from the raw events DataFrame for each hour group based on the counts DataFrame.

    :param all_defective: DataFrame containing raw events with an hour and datetime column.
    :param targets: DataFrame containing the count of events per hour.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param datetime_column: Name of the column containing datetime values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with the selected events.
    """
    # Ensure the datetime column is of datetime type
    all_defective[datetime_column] = pd.to_datetime(all_defective[datetime_column])

    # Merge the raw_df and counts_df on the date + hour column
    merged_df = all_defective.merge(targets, on=[date_column, hour_column], how='left')

    # Initialize an empty DataFrame to store the selected events
    selected_events = pd.DataFrame(columns=all_defective.columns)

    # Iterate over each date+hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target sum for this date + hour
        target_sum = group['defect_count'].iloc[0]

    # Iterate over each hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target count for this hour
        target_count = group[count_column].iloc[0]

        # Sort the group by the datetime column
        sorted_group = group.sort_values(by=datetime_column)

        # Select the first n events
        selected_n_events = sorted_group.head(target_count)

        # Append the selected events to the result DataFrame
        selected_events = pd.concat([selected_events, selected_n_events])

    # Drop the count column from the result DataFrame
    #selected_events = selected_events.drop(columns=[count_column])

    return selected_events


In [ ]:
# Select certain defectives and transform them to rectified events
rectified_events = pick_rectified_events(defectives, targets_rectified)
rectified_events.drop(columns=['Hour', 'Rectified'], inplace=True)
rectified_events['evn'] = 'rectified'
rectified_events['defective_count'] = 0
rectified_events['defect_count'] = 0
rectified_events['rectified_count'] = 1



<ipython-input-77-ee0b19bcb418>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_events = pd.concat([selected_events, selected_n_events])


In [ ]:
rectified_events.head()

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:05:34,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,rectified,HV5ENEBM,0,0,0,0,1,0,0
1,2024-07-26 02:09:28,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,L,rectified,UP6DL72E,0,0,0,0,1,0,0
2,2024-07-26 02:14:40,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,rectified,150Z50UR,0,0,0,0,1,0,0
9,2024-07-26 03:00:12,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,L,rectified,YEFBZSQ2,0,0,0,0,1,0,0
10,2024-07-26 03:18:37,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,M,rectified,HSIZMJ16,0,0,0,0,1,0,0


In [ ]:
# Function to increment a timestamp by 10-20 seconds randomly
def increment_timestamp(timestamp):
    random_seconds = random.randint(1, 5)
    return timestamp + timedelta(seconds=random_seconds)

In [ ]:
# Apply the function to the 'timestamp' column
rectified_events['evt_gmt'] = rectified_events['evt_gmt'].apply(increment_timestamp)

In [ ]:
# Reorder the data columns
synthetic_rectified = rectified_events.reindex(columns=rectified.columns)


In [ ]:
synthetic_rectified.rename(columns={'Date': 'factory_date'}, inplace=True)
#synthetic_rectified.to_csv('synthetic_rectified_regulated.csv', index=False)

In [ ]:
# Change station_id manually
synthetic_rectified['stp'] = synthetic_rectified['stp'] +  '-station2'


In [ ]:
synthetic_rectified.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'evn', 'gid', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count'],
      dtype='object')

**REJECT**

In [ ]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)

ts_df_reject = pd.DataFrame()

for i in range(len(date_list)):
  ts_df_reject= pd.concat([ts_df_reject, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_df_reject.reset_index(drop=True, inplace=True)
ts_df_reject.head()

,ts
0,2024-07-26 02:00:00
1,2024-07-26 02:00:04
2,2024-07-26 02:00:08
3,2024-07-26 02:00:10
4,2024-07-26 02:00:12


In [ ]:
# Import the pretrained synthesizer for Reject
reject_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/reject_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# Generate some instances (to teh same length as timestamps)
synthetic_reject = reject_synthesizer.sample(num_rows=len(ts_df_reject))

Sampling rows: 100%|██████████| 71916/71916 [00:08<00:00, 8484.48it/s]


In [ ]:
# Apply factory_date logic to evt_gmt
synthetic_reject["evt_gmt"]= ts_df_reject["ts"]

synthetic_reject['evt_gmt'] = pd.to_datetime(synthetic_reject['evt_gmt'])
synthetic_reject['factory_date'] = synthetic_reject['evt_gmt'].apply(get_factory_date)

In [ ]:
synthetic_reject.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-vesfmW,reject,0,0,0,1,0,0,0
1,2024-07-26 02:00:04,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-tRzFCM,reject,0,0,0,1,0,0,0
2,2024-07-26 02:00:08,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-zMJmno,reject,0,0,0,1,0,0,0
3,2024-07-26 02:00:10,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-nATVMQ,reject,0,0,0,1,0,0,0
4,2024-07-26 02:00:12,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-ZNygiF,reject,0,0,0,1,0,0,0


In [ ]:
# Generate gid
synthetic_reject["gid"]= [generate_random_string() for _ in range(len(synthetic_reject))]

In [ ]:
synthetic_reject['factory_time'] = synthetic_reject['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_reject.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,RMJHB2J5,reject,0,0,0,1,0,0,0,27000
1,2024-07-26 02:00:04,2024-07-26,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,FO6INB2R,reject,0,0,0,1,0,0,0,27004
2,2024-07-26 02:00:08,2024-07-26,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,L5TGEDZH,reject,0,0,0,1,0,0,0,27008
3,2024-07-26 02:00:10,2024-07-26,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,JBR8S0R5,reject,0,0,0,1,0,0,0,27010
4,2024-07-26 02:00:12,2024-07-26,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,IKVS3T68,reject,0,0,0,1,0,0,0,27012


In [ ]:
# Perform a forward-looking merge
synthetic_reject = synthetic_reject.sort_values('factory_time')
synthetic_reject = pd.merge_asof(synthetic_reject, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_reject.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
synthetic_reject = synthetic_reject.rename(columns={'hour': 'Hour', 'factory_date': 'Date'})
synthetic_reject.head()




,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour
0,2024-07-26 02:00:00,2024-07-26,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,RMJHB2J5,reject,0,0,0,1,0,0,0,1
1,2024-07-29 02:00:00,2024-07-29,sdv-id-YORsUx,1,Morning,1,mas,1,kreeda,ABC001,...,RZNWDKYL,reject,0,0,0,1,0,0,0,1
2,2024-07-31 02:00:00,2024-07-31,sdv-id-nvsSin,1,Morning,1,mas,1,kreeda,ABC001,...,738WFGN1,reject,0,0,0,1,0,0,0,1
3,2024-07-28 02:00:00,2024-07-28,sdv-id-TBywNh,1,Morning,1,mas,1,kreeda,ABC001,...,7GBR3EP6,reject,0,0,0,1,0,0,0,1
4,2024-08-01 02:00:00,2024-08-01,sdv-id-bjvTsu,1,Morning,1,mas,1,kreeda,ABC001,...,8G6IKSAB,reject,0,0,0,1,0,0,0,1


In [ ]:
# Target for defectives
targets_reject = get_relevant_target(["Date", "Hour", "Reject"])
targets_reject

,Date,Hour,Reject
0,2024-07-26,1,0
1,2024-07-26,2,2
2,2024-07-26,3,2
3,2024-07-26,4,1
4,2024-07-26,5,2
...,...,...,...
72,2024-08-01,7,0
73,2024-08-01,8,0
74,2024-08-01,9,1
75,2024-08-01,10,1


In [ ]:
# Adjust the reject count to tally with the given target
reject_adjusted = adjust_hourly_counts(synthetic_reject, targets_reject,
                                          "Reject", hour_column="Hour")
reject_adjusted.head(20)

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour,Reject
0,2024-07-26 03:15:08,2024-07-26,sdv-id-BLAaHl,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,2,2
1,2024-07-26 03:19:10,2024-07-26,sdv-id-xsgEyw,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,2,2
2,2024-07-26 04:31:31,2024-07-26,sdv-id-FBqNhY,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,3,2
3,2024-07-26 04:48:21,2024-07-26,sdv-id-ebkSQr,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,3,2
4,2024-07-26 05:16:32,2024-07-26,sdv-id-LEeFpi,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,4,1
5,2024-07-26 07:13:50,2024-07-26,sdv-id-OKmCqV,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,5,2
6,2024-07-26 07:24:09,2024-07-26,sdv-id-TdItjx,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,5,2
7,2024-07-26 07:53:30,2024-07-26,sdv-id-VsiWyf,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,6,2
8,2024-07-26 08:07:12,2024-07-26,sdv-id-qedvDm,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,6,2
9,2024-07-26 08:42:29,2024-07-26,sdv-id-BcQwpg,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,7,2


In [ ]:
reject_adjusted.rename(columns={'Date': 'factory_date'}, inplace=True)
synthetic_reject_regulated= reject_adjusted.reindex(columns=reject.columns)
synthetic_reject_regulated.head()



,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 03:15:08,2024-07-26,sdv-id-BLAaHl,1,Morning,1,mas,1,kreeda,ABC001,...,XL,KDFGSQ2P,reject,0,0,0,1,0,0,0
1,2024-07-26 03:19:10,2024-07-26,sdv-id-xsgEyw,1,Morning,1,mas,1,kreeda,ABC001,...,XL,IKOXSVN7,reject,0,0,0,1,0,0,0
2,2024-07-26 04:31:31,2024-07-26,sdv-id-FBqNhY,1,Morning,1,mas,1,kreeda,ABC001,...,L,1U830GMI,reject,0,0,0,1,0,0,0
3,2024-07-26 04:48:21,2024-07-26,sdv-id-ebkSQr,1,Morning,1,mas,1,kreeda,ABC001,...,XL,8IBMB7EO,reject,0,0,0,1,0,0,0
4,2024-07-26 05:16:32,2024-07-26,sdv-id-LEeFpi,1,Morning,1,mas,1,kreeda,ABC001,...,XL,WY7X3K54,reject,0,0,0,1,0,0,0


In [ ]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory","unit_id", "unit", "building_id",
                        "building","department_id","department","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "module_id","module", "stp", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

edit_df_bctx_org(synthetic_reject_regulated, editable_columns)

edit_value(synthetic_reject_regulated, "reject_count", 1)

In [ ]:
synthetic_reject_regulated.head(10)

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-07-26 03:15:08,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,KDFGSQ2P,reject,0,0,0,1,0,0,0
1,2024-07-26 03:19:10,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,IKOXSVN7,reject,0,0,0,1,0,0,0
2,2024-07-26 04:31:31,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,L,1U830GMI,reject,0,0,0,1,0,0,0
3,2024-07-26 04:48:21,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,8IBMB7EO,reject,0,0,0,1,0,0,0
4,2024-07-26 05:16:32,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,WY7X3K54,reject,0,0,0,1,0,0,0
5,2024-07-26 07:13:50,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,L,7N656FUV,reject,0,0,0,1,0,0,0
6,2024-07-26 07:24:09,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,M,S99PD7ZJ,reject,0,0,0,1,0,0,0
7,2024-07-26 07:53:30,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,TUNI3ENB,reject,0,0,0,1,0,0,0
8,2024-07-26 08:07:12,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,AEKH1HZ2,reject,0,0,0,1,0,0,0
9,2024-07-26 08:42:29,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,XL,KPXYS1MR,reject,0,0,0,1,0,0,0


In [ ]:
#synthetic_reject_regulated.to_csv('synthetic_reject_regulated.csv', index=False)

In [ ]:
synthetic_reject_regulated['stp'] = synthetic_reject_regulated['stp'] +  '-station2'

In [ ]:
synthetic_reject_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'gid', 'evn', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count'],
      dtype='object')

**MANPOWER**

Generate categorical data columns using for example, ftt events and generate the numbers directly using schedule.

In [ ]:
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [ ]:
def expand_schedule_with_dates(schedule_df, date_list):
    """
    Repeat the schedule DataFrame for each date in the date_list and add the relevant date.

    :param schedule_df: The schedule DataFrame with 11 hours.
    :param date_list: List of dates to repeat the schedule for.
    :return: A DataFrame with the schedule repeated for each date, with the date added to each row.
    """
    # Create an empty DataFrame to store the results
    expanded_df = pd.DataFrame()

    # Repeat the schedule DataFrame for each date
    for date in date_list:
        temp_df = schedule_df.copy()  # Copy the schedule DataFrame
        temp_df['Date'] = date  # Add the date column
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

    return expanded_df

In [ ]:
new_schedule = expand_schedule_with_dates(schedule, date_list)
#new_schedule

In [ ]:
# Just using synthetic_ftt_regulated dataframe to extract the underlying org
manpower_cat = synthetic_ftt_regulated.head(len(new_schedule))
manpower_cat.drop(columns=['plan_id', 'evn', 'ftt_count', 'defective_count', 'defect_count',
                           'reject_count', 'rectified_count', 'input_count',
                           'rw_defective_count', 'buyer_id','buyer','vendor_id',
                           'vendor',	'po_id'	,'po',	'so_id',	'so',	'li_id'	,
                           'li',	'fabric_type_id',	'fabric_type',	'product_category_id',
                           'product_category',	'product_type_id',	'product_type',
                           'style_id',	'style',	'color_id',	'color'	,'size_id',	'size'], inplace=True)


In [ ]:
# Iterate over each datetime object in the Series and format it
start_time_str = [dt.strftime('%H:%M:%S') for dt in new_schedule["start_time"]]

ts_manpower = pd.DataFrame()
ts_manpower['start_time'] = start_time_str
ts_manpower





,start_time
0,07:30:00
1,08:30:00
2,09:30:00
3,10:30:00
4,11:30:00
...,...
72,14:00:00
73,15:00:00
74,16:00:00
75,17:00:00


In [ ]:
# Generate timestamps for manpower events
manpower_cat['evt_gmt'] = pd.to_datetime(new_schedule['Date'].astype(str) + ' ' +ts_manpower['start_time'] )
#manpower_cat

In [ ]:
# Generate Hour
manpower_cat['Hour'] = new_schedule['hour']
#manpower_cat


In [ ]:
# Target for defectives (This example target does not have ironman and qc)
targets_manpower = get_relevant_target(["Date", "Hour", "Operator", "Helper", "Man power"
                                        #"Iron man", "QC"
                                        ])
targets_manpower

,Date,Hour,Operator,Helper,Man power
0,2024-07-26,1,23,5,28
1,2024-07-26,2,23,5,28
2,2024-07-26,3,23,5,28
3,2024-07-26,4,23,5,28
4,2024-07-26,5,23,5,28
...,...,...,...,...,...
72,2024-08-01,7,26,8,34
73,2024-08-01,8,26,8,34
74,2024-08-01,9,26,8,34
75,2024-08-01,10,26,8,34


In [ ]:
# lookup the hourly values from given target
manpower_cat.rename(columns={'factory_date': 'Date'}, inplace=True)
synthetic_manpower = manpower_cat.merge(targets_manpower, on=["Date", 'Hour'], how='left')
synthetic_manpower.drop(columns=['Hour'], inplace=True)
synthetic_manpower.rename(columns={'Operator': 'operator', 'Helper': 'helper', 'Man power': 'manpower'}, inplace=True)
synthetic_manpower

,evt_gmt,Date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,station_id,station,hsk,buf,apt,stp,stpv,operator,helper,manpower
0,2024-07-26 07:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
1,2024-07-26 08:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
2,2024-07-26 09:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
3,2024-07-26 10:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
4,2024-07-26 11:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
73,2024-08-01 15:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
74,2024-08-01 16:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
75,2024-08-01 17:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28


In [ ]:
# Add missing column and values if not present in target
if "iron man" not in synthetic_manpower.columns:
    synthetic_manpower["ironman"] = 0
if "qc" not in synthetic_manpower.columns:
    synthetic_manpower["qc"] = 0

In [ ]:
synthetic_manpower.rename(columns={'Date': 'factory_date'}, inplace=True)

In [ ]:
synthetic_manpower

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,operator,helper,manpower,ironman,qc
0,2024-07-26 07:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
73,2024-08-01 15:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
74,2024-08-01 16:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
75,2024-08-01 17:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0


In [ ]:
# edit bctx and values as needed
editable_columns = ['shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department', 'floor_id', 'floor',
       'zone_id', 'zone', 'section_id', 'section', 'module_id', 'module',"stp"
       ]

edit_df_bctx_org(synthetic_manpower, editable_columns)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,operator,helper,manpower,ironman,qc
0,2024-07-26 07:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
73,2024-08-01 15:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
74,2024-08-01 16:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
75,2024-08-01 17:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0


In [ ]:
manpower

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,2024-06-01 02:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,25,23,1,1,0


In [ ]:
# Order the columns
synthetic_manpower = synthetic_manpower.reindex(columns=manpower.columns)
synthetic_manpower


,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,2024-07-26 07:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
1,2024-07-26 08:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
2,2024-07-26 09:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
3,2024-07-26 10:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
4,2024-07-26 11:30:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-08-01 14:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
73,2024-08-01 15:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
74,2024-08-01 16:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
75,2024-08-01 17:00:00,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0


In [ ]:
# Manually change station_id
synthetic_manpower['stp'] = synthetic_manpower['stp'] +  '-station2'


In [ ]:
synthetic_manpower.columns

Index(['evt_gmt', 'factory_date', 'shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'unit_id', 'unit', 'department_id', 'department',
       'floor_id', 'floor', 'zone_id', 'zone', 'section_id', 'section',
       'module_id', 'module', 'station_id', 'station', 'hsk', 'buf', 'apt',
       'stp', 'stpv', 'manpower', 'operator', 'helper', 'ironman', 'qc'],
      dtype='object')

In [ ]:
#synthetic_manpower.to_csv('synthetic_manpower_regulated.csv', index=False)

**Now we are going to generate an eid for all synthetic data**

In [ ]:
def add_unique_event_ids_and_save(*dfs, filenames):
    """
    Add unique event_id column to multiple DataFrames and save each to a CSV file.

    Parameters:
    *dfs : list of pd.DataFrame
        List of DataFrames to which unique event_ids will be added.
    filenames : list of str
        List of filenames corresponding to each DataFrame for saving as CSV.

    Returns:
    list of pd.DataFrame
        List of DataFrames with unique event_ids added.
    """

    # Calculate the total number of rows across all DataFrames
    total_rows = sum(len(df) for df in dfs)

    # Generate unique event_ids
    event_ids = [str(uuid.uuid4()) for _ in range(total_rows)]

    # Initialize the start index for slicing event_ids
    start_idx = 0

    # Function to move the last column to the first position
    def move_last_column_to_first(df):
        cols = df.columns.tolist()
        new_cols = [cols[-1]] + cols[:-1]
        return df[new_cols]

    # Assign unique event_ids to each DataFrame and save to CSV
    for df, filename in zip(dfs, filenames):
        end_idx = start_idx + len(df)
        df['eid'] = event_ids[start_idx:end_idx]
        start_idx = end_idx
        df = move_last_column_to_first(df)
        df.to_csv(filename, index=False)





# Filenames for saving the CSVs
filenames = ['regulated_synthetic_ftt.csv', 'regulated_synthetic_defective.csv',
             'regulated_synthetic_defect.csv', 'regulated_synthetic_rectified.csv',
             'regulated_synthetic_reject.csv','regulated_synthetic_manpower.csv']



# Add unique event_ids to DataFrames and save to CSVs
add_unique_event_ids_and_save(synthetic_ftt_regulated,synthetic_defective_regulated,
                              synthetic_defects,synthetic_rectified,
                              synthetic_reject_regulated,synthetic_manpower
                              ,filenames=filenames)



In [ ]:
# Adding plan to a seperate csv file
plan.to_csv('plan.csv', index=False)

In [ ]:
# Function to bring the added eid to the very begining
def move_last_column_to_front(df):
    """
    Moves the last column of the DataFrame to the front.

    Parameters:
    df (pd.DataFrame): The DataFrame to be processed.

    Returns:
    pd.DataFrame: The DataFrame with the last column moved to the front.
    """
    # Get the last column name
    last_column = df.columns[-1]

    # Reorder columns with the last column first
    new_order = [last_column] + list(df.columns[:-1])

    # Reindex the DataFrame
    df = df[new_order]

    return df


In [ ]:
# Drop gid if present and combine ftt,reject,defective and rectified as production data


def generate_production_dataset(df_list, column_name):
    """
    Checks if a particular column exists in each DataFrame in the list,
    drops that column if it exists, and then merges all DataFrames vertically.

    Parameters:
    df_list (list of pd.DataFrame): List of DataFrames to be processed.
    column_name (str): The name of the column to check and drop.

    Returns:
    pd.DataFrame: The merged DataFrame.
    """
    processed_dfs = []

    for df in df_list:
        if column_name in df.columns:
            df = df.drop(columns=[column_name])
        processed_dfs.append(df)

    # Concatenate all DataFrames vertically
    merged_df = pd.concat(processed_dfs, ignore_index=True)
    merged_df = move_last_column_to_front(merged_df)

    merged_df.to_csv('production.csv', index=False)



In [ ]:
# Generate production dataset

df_list = [synthetic_ftt_regulated,synthetic_defective_regulated,
           synthetic_rectified,synthetic_reject_regulated]
column_name = 'gid'
generate_production_dataset(df_list, column_name)

<ipython-input-123-f38f3c0c3982>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(processed_dfs, ignore_index=True)


**REJECT REASON**

In [151]:
synthetic_reject_regulated.columns

Index(['evt_gmt', 'factory_date', 'plan_id', 'shift_id', 'shift', 'tenant_id',
       'tenant', 'cluster_id', 'cluster', 'factory_id', 'factory',
       'building_id', 'building', 'unit_id', 'unit', 'department_id',
       'department', 'floor_id', 'floor', 'zone_id', 'zone', 'section_id',
       'section', 'module_id', 'module', 'station_id', 'station', 'hsk', 'buf',
       'apt', 'stp', 'stpv', 'buyer_id', 'buyer', 'vendor_id', 'vendor',
       'po_id', 'po', 'so_id', 'so', 'li_id', 'li', 'fabric_type_id',
       'fabric_type', 'product_category_id', 'product_category',
       'product_type_id', 'product_type', 'style_id', 'style', 'color_id',
       'color', 'size_id', 'size', 'gid', 'evn', 'ftt_count',
       'defective_count', 'defect_count', 'reject_count', 'rectified_count',
       'input_count', 'rw_defective_count', 'eid'],
      dtype='object')

In [152]:
rejects_for_rr = synthetic_reject_regulated.copy()
rejects_for_rr.drop(columns=['evn', 'ftt_count','defective_count',
                             'defect_count', 'reject_count', 'rectified_count',
                             'input_count', 'rw_defective_count'], inplace=True)
rejects_for_rr.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,product_type_id,product_type,style_id,style,color_id,color,size_id,size,gid,eid
0,2024-07-26 03:15:08,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Top,1,3667M,1,Print,1,XL,KDFGSQ2P,aee90fe4-f80f-421e-b8ed-47e0a38fd370
1,2024-07-26 03:19:10,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Top,1,3667M,1,Print,1,XL,IKOXSVN7,18dabb96-8991-4d74-af78-d5eff1030b28
2,2024-07-26 04:31:31,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Top,1,3667M,1,Print,1,L,1U830GMI,99c5c059-bb61-492b-8b9f-83587680ee4f
3,2024-07-26 04:48:21,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Top,1,3667M,1,Print,1,XL,8IBMB7EO,0133bc34-a79e-4fe4-af39-cbcf0a00fe8c
4,2024-07-26 05:16:32,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Top,1,3667M,1,Print,1,XL,WY7X3K54,197c2683-b853-4547-ae4f-8c7366a41833


In [153]:
# Import the pretrained synthesizer for reject reasons
rr_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/rr_synthesizer.pkl'
)

In [154]:
# Generate a set of synthetic reject reasons
synthetic_rr = rr_synthesizer.sample(num_rows=len(rejects_for_rr))


Sampling rows: 100%|██████████| 132/132 [00:00<00:00, 3668.17it/s]


In [155]:
synthetic_rr_regulated = pd.concat([rejects_for_rr, synthetic_rr], axis=1)
synthetic_rr_regulated.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,color_id,color,size_id,size,gid,eid,sign,reject_reason_id,reject_reason_code,reject_reason
0,2024-07-26 03:15:08,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Print,1,XL,KDFGSQ2P,aee90fe4-f80f-421e-b8ed-47e0a38fd370,1,5,304,Fabric Defect
1,2024-07-26 03:19:10,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Print,1,XL,IKOXSVN7,18dabb96-8991-4d74-af78-d5eff1030b28,1,5,304,Fabric Defect
2,2024-07-26 04:31:31,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Print,1,L,1U830GMI,99c5c059-bb61-492b-8b9f-83587680ee4f,1,2,301,Uneven
3,2024-07-26 04:48:21,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Print,1,XL,8IBMB7EO,0133bc34-a79e-4fe4-af39-cbcf0a00fe8c,1,4,303,Needle Defect
4,2024-07-26 05:16:32,2024-07-26,12f0ffe7-c509-49b5-b203-83be908b2881,1,Morning,1,mas,1,kreeda,ABC001,...,1,Print,1,XL,WY7X3K54,197c2683-b853-4547-ae4f-8c7366a41833,1,9,308,Other


In [156]:
eid_column = synthetic_rr_regulated['eid']

In [157]:
synthetic_rr_regulated = synthetic_rr_regulated.reindex(columns=reject_reason.columns)
synthetic_rr_regulated = pd.concat([eid_column, synthetic_rr_regulated], axis=1)
synthetic_rr_regulated.head()

,eid,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,style,color_id,color,size_id,size,sign,gid,reject_reason_id,reject_reason_code,reject_reason
0,aee90fe4-f80f-421e-b8ed-47e0a38fd370,2024-07-26 03:15:08,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,...,3667M,1,Print,1,XL,1,KDFGSQ2P,5,304,Fabric Defect
1,18dabb96-8991-4d74-af78-d5eff1030b28,2024-07-26 03:19:10,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,...,3667M,1,Print,1,XL,1,IKOXSVN7,5,304,Fabric Defect
2,99c5c059-bb61-492b-8b9f-83587680ee4f,2024-07-26 04:31:31,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,...,3667M,1,Print,1,L,1,1U830GMI,2,301,Uneven
3,0133bc34-a79e-4fe4-af39-cbcf0a00fe8c,2024-07-26 04:48:21,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,...,3667M,1,Print,1,XL,1,8IBMB7EO,4,303,Needle Defect
4,197c2683-b853-4547-ae4f-8c7366a41833,2024-07-26 05:16:32,2024-07-26,1,Morning,1,mas,1,kreeda,ABC001,...,3667M,1,Print,1,XL,1,WY7X3K54,9,308,Other


In [158]:
synthetic_rr_regulated.to_csv('regulated_synthetic_rr.csv', index=False)